In [1]:

# Import Liabraries:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import urllib

# Sentiment analysis

The file *IMD.txt* contains reviews by users about movies, organized by line (each line in the file is a different review). In the *labelled.txt*, for each review we have a label (positive/negative).

We want to find out if the review is positive or negative.
Let's try to train a rnn for reviews classification.


## Preparing the datasets
First of all, we have to prepare our datasets.

In [73]:
size_of_dataset = 10000

### Reading the files and converting them into Tensors

In [2]:
# Load reviews to string
IMDB_reviews_path = "IMD.txt"
with open(IMDB_reviews_path, encoding='utf8') as f:
    IMDB_reviews = f.read()

In [3]:
type(IMDB_reviews)

str

In [4]:
len(IMDB_reviews)

52451852

Now, we have to remove all puntuaction simbols inside the sentences. 

In [6]:
# Remove punctuaction
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [7]:
list_characters = [c for c in IMDB_reviews if c not in punctuation]

In [8]:
len(IMDB_reviews) - len(list_characters)

2104575

In [9]:
IMDB_reviews = "".join(list_characters)

In [10]:
IMDB_reviews[:1000]

'I grew up watching and loving the Thunderbirds All my mates at school watched We played Thunderbirds before school during lunch and after school We all wanted to be Virgil or Scott No one wanted to be Alan Counting down from 5 became an art form I took my children to see the movie hoping they would get a glimpse of what I loved as a child How bitterly disappointing The only high point was the snappy theme tune Not that it could compare with the original score of the Thunderbirds Thankfully early Saturday mornings one television channel still plays reruns of the series Gerry Anderson and his wife created Jonatha Frakes should hand in his directors chair his version was completely hopeless A waste of film Utter rubbish A CGI remake may be acceptable but replacing marionettes with Homo sapiens subsp sapiens was a huge error of judgment\nWhen I put this movie in my DVD player and sat down with a coke and some chips I had some expectations I was hoping that this movie would contain some of

In [11]:
# split the reviews
IMDB_reviews = IMDB_reviews.split("\n")

now `reviews` is a list of sentences:

In [12]:
# each element of reviews is a review
IMDB_reviews[0]
IMDB_reviews = IMDB_reviews[:size_of_dataset]

In [13]:
len(IMDB_reviews)

10000

In [14]:
print(IMDB_reviews[9])

My Super Ex Girlfriend turned out to be a pleasant surprise for me I was really expecting a horrible movie that would probably be stupid and predictable and you know what It was But this movie did have so many wonderful laughs and a fun plot that anyone could get a kick out of I know that this was a very cheesy movie but Uma and Anna were just so cool and Steve was such a great addition along with a great cast that looked like they had so much fun and thats what made the movie really workbr br Jenny Johnsonscary thats my best friends actual name is not your typical average librarian looking woman when Matt your average male asks her out hes in for more than he expected hes asked GGirl out on a date the super hero of the world But when he finds out what a jealous and crazy girl she really is and decides that it may be a good idea that they spend some time apart but Jenny wont have it since hes fallen for another girl Hannah and she will make his life a living hell I mean lets face it he

In [15]:
#last elem in reviews list is an empty string, so discard it
#print(IMDB_reviews[40000])
#IMDB_reviews = IMDB_reviews[:-1]

Since the input of our model will be the individual word, now we have to split the words inside the sentence

In [16]:
IMDB_reviews = [[w for w in r.split(" ") if len(w)>0] for r in IMDB_reviews]

now `reviews` is a list of list. Each reaview is a list of words.

In [17]:
IMDB_reviews[0]

['I',
 'grew',
 'up',
 'watching',
 'and',
 'loving',
 'the',
 'Thunderbirds',
 'All',
 'my',
 'mates',
 'at',
 'school',
 'watched',
 'We',
 'played',
 'Thunderbirds',
 'before',
 'school',
 'during',
 'lunch',
 'and',
 'after',
 'school',
 'We',
 'all',
 'wanted',
 'to',
 'be',
 'Virgil',
 'or',
 'Scott',
 'No',
 'one',
 'wanted',
 'to',
 'be',
 'Alan',
 'Counting',
 'down',
 'from',
 '5',
 'became',
 'an',
 'art',
 'form',
 'I',
 'took',
 'my',
 'children',
 'to',
 'see',
 'the',
 'movie',
 'hoping',
 'they',
 'would',
 'get',
 'a',
 'glimpse',
 'of',
 'what',
 'I',
 'loved',
 'as',
 'a',
 'child',
 'How',
 'bitterly',
 'disappointing',
 'The',
 'only',
 'high',
 'point',
 'was',
 'the',
 'snappy',
 'theme',
 'tune',
 'Not',
 'that',
 'it',
 'could',
 'compare',
 'with',
 'the',
 'original',
 'score',
 'of',
 'the',
 'Thunderbirds',
 'Thankfully',
 'early',
 'Saturday',
 'mornings',
 'one',
 'television',
 'channel',
 'still',
 'plays',
 'reruns',
 'of',
 'the',
 'series',
 'Gerry',

In [18]:
# compute the length of each review
IMDB_reviews_lens = [len(r) for r in IMDB_reviews]
# print the lenght of the first x reviews

In [19]:
# print the average review lenght
sum(IMDB_reviews_lens) / len(IMDB_reviews_lens)

230.0499

Our rnn cannot use words directly, we have to convert each word in a number. When we pass a review to the model, the model will see a sequence of numbers.
To do this, let's create the vocabulary of our words.

In [20]:
# build vocabulary
words = list(set([w for r in IMDB_reviews for w in r]))
vocab = {words[i]: i+1 for i in range(len(words))} # we reserve i=0 for pad sequences

In [21]:
len(vocab)

87170

In [22]:
vocab

{'trailer': 1,
 'Morphin': 2,
 'nuit': 3,
 'Paternity': 4,
 'standpoints': 5,
 'mumbojumbo': 6,
 'lecturesneed': 7,
 'MrFuji': 8,
 'forget': 9,
 'careerbr': 10,
 'tempted': 11,
 'inquirybr': 12,
 'madeall': 13,
 'Nara': 14,
 'movieThe': 15,
 'macabrelooking': 16,
 'crystalline': 17,
 'Christie': 18,
 'modernity': 19,
 'GOODFELLAS': 20,
 'partsthe': 21,
 'Valencia': 22,
 'forcefed': 23,
 'De': 24,
 'skinned': 25,
 'altogetherbr': 26,
 'kingsley': 27,
 'fightin': 28,
 'Xfiles': 29,
 'kidneys': 30,
 'Buonovolonta': 31,
 'lavender': 32,
 'Lamarrmeets': 33,
 'facechange': 34,
 'Lasek': 35,
 'Slugs': 36,
 'inheriting': 37,
 'ISLAND': 38,
 'Gordo': 39,
 'IMAGES': 40,
 'nauseatingly': 41,
 '1898': 42,
 'restsbr': 43,
 'ringbr': 44,
 'Thurig': 45,
 'Mosely': 46,
 'Hood': 47,
 '½br': 48,
 'sporting': 49,
 'Behavior': 50,
 'ventriloquist': 51,
 'courage': 52,
 'Eurowestern': 53,
 'realized': 54,
 'bribery': 55,
 'mushroom': 56,
 'Darbys': 57,
 'series\x97all': 58,
 'Cannibal': 59,
 'Maniacs': 60,

In [23]:
# Convert reviews to word indexes
IMDB_reviews = [[vocab[w] for w in r] for r in IMDB_reviews]

In [24]:
IMDB_reviews[0]

[28964,
 30316,
 51885,
 8355,
 11762,
 79655,
 6747,
 64110,
 13033,
 83392,
 87108,
 86508,
 69440,
 75406,
 18397,
 80878,
 64110,
 32924,
 69440,
 15640,
 43502,
 11762,
 87125,
 69440,
 18397,
 73769,
 10400,
 24497,
 6255,
 34393,
 22358,
 14249,
 32858,
 38749,
 10400,
 24497,
 6255,
 363,
 22098,
 57263,
 9694,
 26528,
 24317,
 942,
 86665,
 45905,
 28964,
 69252,
 83392,
 73616,
 24497,
 44901,
 6747,
 6197,
 57782,
 25357,
 73602,
 25128,
 15944,
 76412,
 84972,
 82934,
 28964,
 17292,
 52805,
 15944,
 64464,
 74733,
 66779,
 83450,
 44820,
 30971,
 7671,
 3568,
 70796,
 6747,
 2084,
 57707,
 55513,
 17741,
 10294,
 20488,
 15407,
 49051,
 58880,
 6747,
 19224,
 64149,
 84972,
 6747,
 64110,
 63270,
 60507,
 77195,
 8969,
 38749,
 33330,
 7041,
 46849,
 32736,
 13310,
 84972,
 6747,
 46351,
 4411,
 14536,
 11762,
 12966,
 19312,
 70514,
 82224,
 23236,
 60257,
 60362,
 57656,
 12966,
 40064,
 5098,
 12966,
 81925,
 70796,
 45656,
 40781,
 62244,
 50707,
 84972,
 36365,
 45091

In [25]:
words[9-1]

'forget'

The reviews have different lengths, while, we want that all reviews have the same length


In [26]:
seq_len = 200    # number of words for each sentence

In [27]:
# Clip reviews to max seq_len words
IMDB_reviews = [r[:seq_len] for r in IMDB_reviews]

In [28]:
# Print average review length now
IMDB_review_lens = [len(r) for r in IMDB_reviews]
print(sum(IMDB_review_lens)/len(IMDB_review_lens))

158.2857


In [29]:
# Pad reviews shorter than seq_len
# TODO test padding at the end
IMDB_reviews = [[0]*(seq_len - len(r)) + r for r in IMDB_reviews]

In [30]:
# Print average review length now
IMDB_review_lens = [len(r) for r in IMDB_reviews]
print(sum(IMDB_review_lens)/len(IMDB_review_lens))

200.0


Now, `reviews` is a list of list; each elem of `reviews` is a list with `seq_len` elements. So we can convert `reviews` in a Tensor:


In [31]:
# Convert reviews to tensor
data = torch.LongTensor(IMDB_reviews)

In [32]:
data.size()

torch.Size([10000, 200])

Finally the data tensor is ready. Now it's the turn of the labels.


In [33]:
# Load labels to string
sentiments_path = "labelled.txt"
with open(sentiments_path, encoding="utf8") as f:
    sentiments = f.read()

In [34]:
sentiments

'negative\nnegative\nnegative\nnegative\npositive\nnegative\npositive\nnegative\npositive\npositive\npositive\nnegative\npositive\nnegative\nnegative\npositive\npositive\nnegative\nnegative\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\npositive\nnegative\npositive\npositive\nnegative\nnegative\npositive\nnegative\nnegative\nnegative\nnegative\nnegative\nnegative\nnegative\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\npositive\npositive\nnegative\npositive\nnegative\npositive\nnegative\nnegative\npositive\npositive\npositive\npositive\npositive\nnegative\npositive\npositive\npositive\nnegative\nnegative\nnegative\nnegative\npositive\nnegative\npositive\nnegative\nnegative\npositive\nnegative\npositive\nnegative\nnegative\nnegative\nnegative\npositive\npositive\npositive\npositive\nnegative\nnegative\nnegative\npositive\npositive\npositive\npositive\npositive\nnegative\npositive\npositive\nnegative\npositive\nnegative\

In [35]:
sentiments = sentiments.split("\n")

In [36]:
sentiments = sentiments[:size_of_dataset]
len(sentiments)

10000

In [37]:
# discard the last elem of sentiments (empty string)
#sentiments = sentiments[:-1]

In [38]:
sentiments = [0 if sentiments[i]=='negative' else 1 for i in range(len(sentiments))]

In [39]:
# Convert sentiments to tensor
labels = torch.LongTensor(sentiments)

In [40]:
labels.size()

torch.Size([10000])

### Defining the Datasets

In [41]:
# some parameters
frac_train = 0.6 # fraction of data for train
frac_val = 0.3   # fraction of data for val
batch_size = 64

In [42]:
# shuffle dataset
num_data = data.size(0)
shuffle_idx = torch.randperm(num_data)
data = data[shuffle_idx,:]
labels = labels[shuffle_idx]

In [43]:
# split training, validation and test
num_train = int(num_data*frac_train)
num_val = int(num_data*frac_val)
num_test = num_data - num_train - num_val
train_data = data[:num_train,:]
train_labels = labels[:num_train]
val_data = data[num_train:num_train+num_val,:]
val_labels = labels[num_train:num_train+num_val]
test_data = data[num_train+num_val:,:]
test_labels = labels[num_train+num_val:]

In [44]:
print(train_data.size())
print(val_data.size())
print(test_data.size())

torch.Size([6000, 200])
torch.Size([3000, 200])
torch.Size([1000, 200])


In [45]:
# create datasets
train_dataset = TensorDataset(train_data, train_labels)
val_dataset = TensorDataset(val_data, val_labels)
test_dataset = TensorDataset(test_data, test_labels)

In [46]:
# print the first elem of train dataset and its label
print(train_dataset[0])

(tensor([50803, 44125, 77195, 42178, 44000, 42178, 44000, 80122, 81351, 42178,
        73102, 80122, 60448, 60448, 55838, 68681, 85463, 53332, 15944, 15335,
        70078, 26563, 72325,  8482, 24497, 25128,  4210, 58880, 12966, 70156,
        11762, 24497,  6255, 15944, 33645, 50533, 24497, 12966, 21773, 75817,
        87125, 15944,  1399, 60571,  9694, 31716, 45638, 38428,  6101, 68909,
        87125,   942, 48091, 58880, 36986, 75817, 67167, 86508, 15944,  7765,
        67265, 40444, 52684, 55963, 15944, 13740, 24497, 31716, 20457, 59162,
        24497, 81811, 15944,  6511, 86508, 28425, 11478, 16525, 11762, 12966,
        52602, 84972, 81192,  7338, 72325, 72983, 57656,  6747, 35173, 37855,
        43801, 43800, 49762, 11762, 24497, 72983, 78603, 45134, 15944, 17589,
        49054, 47885, 12537,  2687, 40426, 47117, 82493,  6747, 78890, 84972,
        28425, 24497, 48358, 11762, 81053, 10036, 56498, 52805, 15944, 37741,
        11436, 52805,  6747, 60828, 70631, 59742, 85269, 12583,

In [47]:
# Create loaders
loaders = {"train": DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  drop_last=True),
           "val":   DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, drop_last=False),
           "test":  DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, drop_last=False)}

## Defining our RNN

The [torch.nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) module:

Basically, the Embedding module translates numbers in vectors. The *translation function* is learned by this module during the training. 

In [48]:
#####
#  torch.nn.Embedding(num_embeddings: int, embedding_dim: int)
#  where:
#   - num_embeddings: int, is the number of inputs (size of our input vocabulary)
#   - embedding_dim: int  is the size of each feature vector (size of the output vector)
#####
test_embedding = nn.Embedding(5, 10)

In [49]:
# example input to embedding: a batch with 2 elements, each elem has length 3
input = torch.LongTensor([[3, 2, 1],[2, 3, 1]])
input.shape

torch.Size([2, 3])

In [50]:
# Print embedding size
output = test_embedding(input)

In [51]:
output 

tensor([[[ 0.7632, -0.9416,  0.2512,  1.0605, -0.2140,  0.5395, -1.3991,
           0.2964, -0.6399, -1.1429],
         [ 0.0536, -0.6288, -0.9686, -0.4789,  1.0826,  0.6954, -0.0454,
           0.9212, -1.1352, -0.1001],
         [ 0.9282, -0.4315, -1.0151,  0.3072, -0.5539,  1.6647, -0.0774,
          -0.7768, -0.8015,  0.8226]],

        [[ 0.0536, -0.6288, -0.9686, -0.4789,  1.0826,  0.6954, -0.0454,
           0.9212, -1.1352, -0.1001],
         [ 0.7632, -0.9416,  0.2512,  1.0605, -0.2140,  0.5395, -1.3991,
           0.2964, -0.6399, -1.1429],
         [ 0.9282, -0.4315, -1.0151,  0.3072, -0.5539,  1.6647, -0.0774,
          -0.7768, -0.8015,  0.8226]]], grad_fn=<EmbeddingBackward0>)

In [52]:
output.shape

torch.Size([2, 3, 10])

In [53]:
# Define model
class Model(nn.Module):
    
    def __init__(self, num_embed, embed_size, rnn_size):
        # params: 
        # num_embed: the number of the input vocabulary
        # embed_size: the size of the feature embedding
        # rnn_size: the number of neurons in the recurrent layer

        # Call parent constructor
        super().__init__()
        # Store values
        self.rnn_size = rnn_size
        # Define modules
        self.embedding = nn.Embedding(len(vocab)+1, embed_size)
        self.rnn = nn.RNNCell(embed_size, rnn_size) #RNNCell represents only a single time step
        self.output = nn.Linear(rnn_size, 2)
        
    def forward(self, x):
        # Embed data
        x = self.embedding(x)
        # Initialize state
        h = torch.zeros(x.shape[0], self.rnn_size).to(x.device.type) # the state of the cell
        
        # Input is: B x T x F
        # Process each time step
        for t in range(x.shape[1]):
            # Input at time t
            x_t = x[:,t,:]
            # Forward RNN and get new state
            h = self.rnn(x_t, h)
        # Classify final state
        x = self.output(h)
        return x

In [54]:
# Setup device
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [55]:
# Model parameters
embed_size = 1024
rnn_size = 256

In [56]:
# Create model
model = Model(len(vocab)+1, embed_size, rnn_size).to(dev)

In [57]:
# Test model output
model.eval()
test_input = train_dataset[0][0].unsqueeze(0).to(dev)
print("Model output size:", model(test_input).size())

Model output size: torch.Size([1, 2])


In [58]:
# Create optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

# Define a loss 
criterion = nn.CrossEntropyLoss()


In [59]:
# Start training
import torch
import torchvision.models as models
from tqdm import tqdm
best_accuracy= 0.00
for epoch in range(20):
    # Initialize accumulators for computing average loss/accuracy
    epoch_loss_sum = {'train': 0, 'val': 0, 'test': 0}
    epoch_loss_cnt = {'train': 0, 'val': 0, 'test': 0}
    epoch_accuracy_sum = {'train': 0, 'val': 0, 'test': 0}
    epoch_accuracy_cnt = {'train': 0, 'val': 0, 'test': 0}
    # Process each split
    for split in ["train", "val", "test"]:
        # Set network mode
        if split == "train":
            model.train()
            torch.set_grad_enabled(True)
        else:
            model.eval()
            torch.set_grad_enabled(False)
        # Process all data in split
        for input, target in tqdm(loaders[split]):
            # Move to device
            input = input.to(dev)
            target = target.to(dev)
            # Reset gradients
            optimizer.zero_grad()
            # Forward
            output = model(input)
            loss = criterion(output, target)
            # Update loss sum
            epoch_loss_sum[split] += loss.item()
            epoch_loss_cnt[split] += 1
            # Compute accuracy
            _,pred = output.max(1)
            correct = pred.eq(target).sum().item()
            accuracy = correct/input.size(0)
            # Update accuracy sum
            epoch_accuracy_sum[split] += accuracy
            epoch_accuracy_cnt[split] += 1
            # Backward and optimize
            if split == "train":
                loss.backward()
                optimizer.step()
    # Compute average epoch loss/accuracy
    print("epoch loss is ")
    print(epoch_loss_cnt["train"])
    avg_train_loss = epoch_loss_sum["train"]/epoch_loss_cnt["train"]
    avg_train_accuracy = epoch_accuracy_sum["train"]/epoch_accuracy_cnt["train"]
    avg_val_loss = epoch_loss_sum["val"]/epoch_loss_cnt["val"]
    avg_val_accuracy = epoch_accuracy_sum["val"]/epoch_accuracy_cnt["val"]
    avg_test_loss = epoch_loss_sum["test"]/epoch_loss_cnt["test"]
    avg_test_accuracy = epoch_accuracy_sum["test"]/epoch_accuracy_cnt["test"]
    print(f"Epoch: {epoch+1}, TrL={avg_train_loss:.4f}, TrA={avg_train_accuracy:.4f},",
                            f"VL={avg_val_loss:.4f}, VA={avg_val_accuracy:.4f}, ",
                            f"TeL={avg_test_loss:.4f}, TeA={avg_test_accuracy:.4f}")
    
   # saving the model:
    #if avg_test_accuracy>best_accuracy:
        #torch.save(model.state_dict(),'best_checkpoint.model')
       # best_accuracy=avg_test_accuracy



  0%|          | 0/93 [00:00<?, ?it/s]

epoch loss is 
93
Epoch: 1, TrL=0.7198, TrA=0.5346, VL=0.6840, VA=0.5580,  TeL=0.6810, TeA=0.5555


  0%|          | 0/93 [00:00<?, ?it/s]

epoch loss is 
93
Epoch: 2, TrL=0.5847, TrA=0.6875, VL=0.7154, VA=0.5700,  TeL=0.7113, TeA=0.5697


  0%|          | 0/93 [00:00<?, ?it/s]

...

epoch loss is 
93
Epoch: 18, TrL=0.0978, TrA=0.9625, VL=1.7058, VA=0.5987,  TeL=1.7587, TeA=0.5668


  0%|          | 0/93 [00:00<?, ?it/s]

epoch loss is 
93
Epoch: 19, TrL=0.0875, TrA=0.9709, VL=1.7375, VA=0.5810,  TeL=1.8482, TeA=0.5289


100%|██████████| 16/16 [00:11<00:00,  1.41it/s]

epoch loss is 
93
Epoch: 20, TrL=0.1110, TrA=0.9587, VL=1.6022, VA=0.6297,  TeL=1.6931, TeA=0.6145


In [60]:
torch.save(model.state_dict(),'best_checkpoint.model')

In [61]:
vocab

{'trailer': 1,
 'Morphin': 2,
 'nuit': 3,
 'Paternity': 4,
 'standpoints': 5,
 'mumbojumbo': 6,
 'lecturesneed': 7,
 'MrFuji': 8,
 'forget': 9,
 'careerbr': 10,
 'tempted': 11,
 'inquirybr': 12,
 'madeall': 13,
 'Nara': 14,
 'movieThe': 15,
 'macabrelooking': 16,
 'crystalline': 17,
 'Christie': 18,
 'modernity': 19,
 'GOODFELLAS': 20,
 'partsthe': 21,
 'Valencia': 22,
 'forcefed': 23,
 'De': 24,
 'skinned': 25,
 'altogetherbr': 26,
 'kingsley': 27,
 'fightin': 28,
 'Xfiles': 29,
 'kidneys': 30,
 'Buonovolonta': 31,
 'lavender': 32,
 'Lamarrmeets': 33,
 'facechange': 34,
 'Lasek': 35,
 'Slugs': 36,
 'inheriting': 37,
 'ISLAND': 38,
 'Gordo': 39,
 'IMAGES': 40,
 'nauseatingly': 41,
 '1898': 42,
 'restsbr': 43,
 'ringbr': 44,
 'Thurig': 45,
 'Mosely': 46,
 'Hood': 47,
 '½br': 48,
 'sporting': 49,
 'Behavior': 50,
 'ventriloquist': 51,
 'courage': 52,
 'Eurowestern': 53,
 'realized': 54,
 'bribery': 55,
 'mushroom': 56,
 'Darbys': 57,
 'series\x97all': 58,
 'Cannibal': 59,
 'Maniacs': 60,

In [62]:
output

tensor([[ 0.6362, -0.4128],
        [-0.1817,  0.6532],
        [-0.1895,  0.8844],
        [ 0.7426, -0.7749],
        [-1.6168,  1.8368],
        [ 1.5653, -2.5945],
        [-1.4136,  0.8154],
        [ 2.4934, -2.8619],
        [-0.5657,  0.6872],
        [-0.2380,  0.7408],
        [ 2.6912, -3.5099],
        [-2.4509,  3.1096],
        [ 0.1184, -0.3295],
        [-3.9675,  4.4429],
        [ 2.9933, -3.4073],
        [ 4.8156, -4.3903],
        [ 1.4397, -1.6035],
        [ 3.7382, -3.4351],
        [-3.0623,  2.6987],
        [ 0.7674, -0.6147],
        [-1.7002,  1.6700],
        [-6.8453,  7.3160],
        [-0.1098, -0.0616],
        [ 0.5523, -0.3001],
        [-0.8122,  1.0753],
        [ 2.0382, -1.8550],
        [ 1.4765, -2.0671],
        [ 1.5752, -2.2614],
        [-3.9260,  3.5019],
        [ 3.4128, -3.2778],
        [-3.1516,  3.7750],
        [ 1.9361, -1.8584],
        [ 1.1808, -0.5731],
        [ 2.8278, -2.9950],
        [ 1.5332, -1.8536],
        [-1.2571,  0

In [63]:
test_labels


tensor([1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
        1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
        0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
        0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1,
        1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
        0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
        0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,

In [246]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.Collecting tensorflow

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=c3612667d97af5602ec32e778e67dd3d58ec4af84b2f1b621f5affb63ee069d7
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built termcolor
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.0
    Uninstalling importlib-metadata-3.10.0:
      Successfully uninstalled importlib-metadata-3.10.0


In [66]:
# Test review
import tensorflow as tf
#checkpoint=torch.load('best_checkpoint.model')
#model = Model(len(vocab)+1, embed_size, rnn_size,).to(dev)
#model.load_state_dict(checkpoint)
model.eval()
review = "i really liked this movie it is great done"
#review = "i didn t like this move"
#review = "that movie is a complete waste of time and resources"
#review = "honestly i preferred the previous movie of that saga"
review = [vocab[w] for w in review.split(" ")]
review = [0]*(seq_len - len(review)) + review
review = torch.LongTensor(review).unsqueeze(0).to(dev)
# Process review
model.eval()
output = model(test_data).cpu()
probabilities = F.softmax(output,1)
_,pred = output.max(1)


res = tf.math.confusion_matrix(test_labels,pred)

In [70]:
print('Confusion_matrix: ',res)

Confusion_matrix:  tf.Tensor(
[[298 217]
 [167 318]], shape=(2, 2), dtype=int32)
